In [1]:
import os
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import MVTecAD_loaddata as mv
import pandas as pd

res = []

def OCSVM(X, x_test, y, y_test):
    tuned_parameters = [
        {'nu': [2**-3, 2**-4, 2**-5, 2**-6, 2**-7], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'accuracy'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
#         scoring='%s_weighted' % score )
          
    clf.fit(X, y)
          
    a = clf.decision_function(x_test)
    b = clf.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
    return result

def model_train(X_train, X_test):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)

    base_model = EfficientNetB2(include_top=False, weights='imagenet')(inputs)
    x = Reshape((8, 8, 1408))(base_model)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    outputs = Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error")

    model.fit(X_train, X_train, epochs=20, batch_size=16, shuffle=True)
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    f_ts = encoder.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], f_tr.shape[1] * f_tr.shape[2] * f_tr.shape[3]))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], f_ts.shape[1] * f_ts.shape[2] * f_ts.shape[3]))
    return x_tr, x_ts

In [2]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 116s 7s/step - loss: 0.1223
Epoch 2/20
14/14 [==============================] - 95s 7s/step - loss: 0.1034
Epoch 3/20
14/14 [==============================] - 94s 7s/step - loss: 0.0987
Epoch 4/20
14/14 [==============================] - 93s 7s/step - loss: 0.0908
Epoch 5/20
14/14 [==============================] - 93s 7s/step - loss: 0.0547
Epoch 6/20
14/14 [==============================] - 93s 7s/step - loss: 0.0177
Epoch 7/20
14/14 [==============================] - 92s 7s/step - loss: 0.0107
Epoch 8/20
14/14 [==============================] - 92s 7s/step - loss: 0.0087
Epoch 9/20
14/14 [==============================] - 93s 7s/step - loss: 0.0079
Epoch 10/20
14/14 [==============================] - 92s 7s/step - loss: 0.0073
Epoch 11/20
14/14 [==============================] - 92s 7s/step - loss: 0.0069
Epoch 12/20
14/14 [==============================] - 93s 7s/step - loss: 0.0066
Epoch 13/20
14/14 [=============================

In [3]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 131s 7s/step - loss: 0.0518
Epoch 2/20
14/14 [==============================] - 99s 7s/step - loss: 0.0454
Epoch 3/20
14/14 [==============================] - 100s 7s/step - loss: 0.0393
Epoch 4/20
14/14 [==============================] - 98s 7s/step - loss: 0.0349
Epoch 5/20
14/14 [==============================] - 98s 7s/step - loss: 0.0323
Epoch 6/20
14/14 [==============================] - 99s 7s/step - loss: 0.0305
Epoch 7/20
14/14 [==============================] - 10907s 838s/step - loss: 0.0288
Epoch 8/20
14/14 [==============================] - 111s 7s/step - loss: 0.0257
Epoch 9/20
14/14 [==============================] - 103s 7s/step - loss: 0.0211
Epoch 10/20
14/14 [==============================] - 102s 7s/step - loss: 0.0180
Epoch 11/20
14/14 [==============================] - 98s 7s/step - loss: 0.0166
Epoch 12/20
14/14 [==============================] - 100s 7s/step - loss: 0.0157
Epoch 13/20
14/14 [===================

In [4]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 127s 7s/step - loss: 0.0895
Epoch 2/20
14/14 [==============================] - 98s 7s/step - loss: 0.0592
Epoch 3/20
14/14 [==============================] - 98s 7s/step - loss: 0.0312
Epoch 4/20
14/14 [==============================] - 98s 7s/step - loss: 0.0267
Epoch 5/20
14/14 [==============================] - 97s 7s/step - loss: 0.0240
Epoch 6/20
14/14 [==============================] - 105s 8s/step - loss: 0.0228
Epoch 7/20
14/14 [==============================] - 99s 7s/step - loss: 0.0222
Epoch 8/20
14/14 [==============================] - 98s 7s/step - loss: 0.0218
Epoch 9/20
14/14 [==============================] - 98s 7s/step - loss: 0.0215
Epoch 10/20
14/14 [==============================] - 98s 7s/step - loss: 0.0210
Epoch 11/20
14/14 [==============================] - 97s 7s/step - loss: 0.0190
Epoch 12/20
14/14 [==============================] - 99s 7s/step - loss: 0.0102
Epoch 13/20
14/14 [============================

In [5]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
18/18 [==============================] - 175s 7s/step - loss: 0.0322
Epoch 2/20
18/18 [==============================] - 126s 7s/step - loss: 0.0231
Epoch 3/20
18/18 [==============================] - 126s 7s/step - loss: 0.0196
Epoch 4/20
18/18 [==============================] - 125s 7s/step - loss: 0.0188
Epoch 5/20
18/18 [==============================] - 126s 7s/step - loss: 0.0184
Epoch 6/20
18/18 [==============================] - 125s 7s/step - loss: 0.0183
Epoch 7/20
18/18 [==============================] - 125s 7s/step - loss: 0.0181
Epoch 8/20
18/18 [==============================] - 125s 7s/step - loss: 0.0181
Epoch 9/20
18/18 [==============================] - 125s 7s/step - loss: 0.0180
Epoch 10/20
18/18 [==============================] - 126s 7s/step - loss: 0.0180
Epoch 11/20
18/18 [==============================] - 126s 7s/step - loss: 0.0179
Epoch 12/20
18/18 [==============================] - 126s 7s/step - loss: 0.0179
Epoch 13/20
18/18 [==================

In [6]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
17/17 [==============================] - 147s 7s/step - loss: 0.0285
Epoch 2/20
17/17 [==============================] - 118s 7s/step - loss: 0.0268
Epoch 3/20
17/17 [==============================] - 117s 7s/step - loss: 0.0261
Epoch 4/20
17/17 [==============================] - 117s 7s/step - loss: 0.0258
Epoch 5/20
17/17 [==============================] - 117s 7s/step - loss: 0.0257
Epoch 6/20
17/17 [==============================] - 118s 7s/step - loss: 0.0256
Epoch 7/20
17/17 [==============================] - 119s 7s/step - loss: 0.0256
Epoch 8/20
17/17 [==============================] - 117s 7s/step - loss: 0.0255
Epoch 9/20
17/17 [==============================] - 118s 7s/step - loss: 0.0254
Epoch 10/20
17/17 [==============================] - 117s 7s/step - loss: 0.0254
Epoch 11/20
17/17 [==============================] - 116s 7s/step - loss: 0.0254
Epoch 12/20
17/17 [==============================] - 117s 7s/step - loss: 0.0254
Epoch 13/20
17/17 [==================

In [7]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
25/25 [==============================] - 218s 7s/step - loss: 0.0731
Epoch 2/20
25/25 [==============================] - 173s 7s/step - loss: 0.0142
Epoch 3/20
25/25 [==============================] - 185s 7s/step - loss: 0.0077
Epoch 4/20
25/25 [==============================] - 175s 7s/step - loss: 0.0053
Epoch 5/20
25/25 [==============================] - 174s 7s/step - loss: 0.0038
Epoch 6/20
25/25 [==============================] - 174s 7s/step - loss: 0.0030
Epoch 7/20
25/25 [==============================] - 175s 7s/step - loss: 0.0026
Epoch 8/20
25/25 [==============================] - 174s 7s/step - loss: 0.0023
Epoch 9/20
25/25 [==============================] - 175s 7s/step - loss: 0.0021
Epoch 10/20
25/25 [==============================] - 177s 7s/step - loss: 0.0019
Epoch 11/20
25/25 [==============================] - 175s 7s/step - loss: 0.0019
Epoch 12/20
25/25 [==============================] - 174s 7s/step - loss: 0.0017
Epoch 13/20
25/25 [==================

In [8]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
16/16 [==============================] - 144s 7s/step - loss: 0.0386
Epoch 2/20
16/16 [==============================] - 110s 7s/step - loss: 0.0178
Epoch 3/20
16/16 [==============================] - 110s 7s/step - loss: 0.0070
Epoch 4/20
16/16 [==============================] - 109s 7s/step - loss: 0.0042
Epoch 5/20
16/16 [==============================] - 110s 7s/step - loss: 0.0031
Epoch 6/20
16/16 [==============================] - 109s 7s/step - loss: 0.0027
Epoch 7/20
16/16 [==============================] - 109s 7s/step - loss: 0.0024
Epoch 8/20
16/16 [==============================] - 109s 7s/step - loss: 0.0022
Epoch 9/20
16/16 [==============================] - 109s 7s/step - loss: 0.0021
Epoch 10/20
16/16 [==============================] - 110s 7s/step - loss: 0.0020
Epoch 11/20
16/16 [==============================] - 111s 7s/step - loss: 0.0019
Epoch 12/20
16/16 [==============================] - 110s 7s/step - loss: 0.0018
Epoch 13/20
16/16 [==================

In [9]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 125s 7s/step - loss: 0.0869
Epoch 2/20
14/14 [==============================] - 98s 7s/step - loss: 0.0482
Epoch 3/20
14/14 [==============================] - 98s 7s/step - loss: 0.0210
Epoch 4/20
14/14 [==============================] - 98s 7s/step - loss: 0.0144
Epoch 5/20
14/14 [==============================] - 98s 7s/step - loss: 0.0124
Epoch 6/20
14/14 [==============================] - 99s 7s/step - loss: 0.0112
Epoch 7/20
14/14 [==============================] - 100s 7s/step - loss: 0.0106
Epoch 8/20
14/14 [==============================] - 98s 7s/step - loss: 0.0099
Epoch 9/20
14/14 [==============================] - 106s 8s/step - loss: 0.0094
Epoch 10/20
14/14 [==============================] - 100s 7s/step - loss: 0.0091
Epoch 11/20
14/14 [==============================] - 98s 7s/step - loss: 0.0087
Epoch 12/20
14/14 [==============================] - 99s 7s/step - loss: 0.0084
Epoch 13/20
14/14 [==========================

In [10]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
17/17 [==============================] - 150s 7s/step - loss: 0.1196
Epoch 2/20
17/17 [==============================] - 120s 7s/step - loss: 0.0639
Epoch 3/20
17/17 [==============================] - 120s 7s/step - loss: 0.0253
Epoch 4/20
17/17 [==============================] - 119s 7s/step - loss: 0.0203
Epoch 5/20
17/17 [==============================] - 121s 7s/step - loss: 0.0191
Epoch 6/20
17/17 [==============================] - 118s 7s/step - loss: 0.0186
Epoch 7/20
17/17 [==============================] - 118s 7s/step - loss: 0.0183
Epoch 8/20
17/17 [==============================] - 118s 7s/step - loss: 0.0180
Epoch 9/20
17/17 [==============================] - 119s 7s/step - loss: 0.0178
Epoch 10/20
17/17 [==============================] - 120s 7s/step - loss: 0.0174
Epoch 11/20
17/17 [==============================] - 119s 7s/step - loss: 0.0170
Epoch 12/20
17/17 [==============================] - 120s 7s/step - loss: 0.0165
Epoch 13/20
17/17 [==================

In [11]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
20/20 [==============================] - 170s 7s/step - loss: 0.0571
Epoch 2/20
20/20 [==============================] - 144s 7s/step - loss: 0.0219
Epoch 3/20
20/20 [==============================] - 143s 7s/step - loss: 0.0092
Epoch 4/20
20/20 [==============================] - 144s 7s/step - loss: 0.0065
Epoch 5/20
20/20 [==============================] - 143s 7s/step - loss: 0.0055
Epoch 6/20
20/20 [==============================] - 144s 7s/step - loss: 0.0050
Epoch 7/20
20/20 [==============================] - 143s 7s/step - loss: 0.0046
Epoch 8/20
20/20 [==============================] - 145s 7s/step - loss: 0.0044
Epoch 9/20
20/20 [==============================] - 144s 7s/step - loss: 0.0042
Epoch 10/20
20/20 [==============================] - 144s 7s/step - loss: 0.0040
Epoch 11/20
20/20 [==============================] - 146s 7s/step - loss: 0.0038
Epoch 12/20
20/20 [==============================] - 143s 7s/step - loss: 0.0037
Epoch 13/20
20/20 [==================

In [12]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
15/15 [==============================] - 144s 7s/step - loss: 0.0173
Epoch 2/20
15/15 [==============================] - 104s 7s/step - loss: 0.0162
Epoch 3/20
15/15 [==============================] - 104s 7s/step - loss: 0.0157
Epoch 4/20
15/15 [==============================] - 103s 7s/step - loss: 0.0155
Epoch 5/20
15/15 [==============================] - 103s 7s/step - loss: 0.0153
Epoch 6/20
15/15 [==============================] - 104s 7s/step - loss: 0.0152
Epoch 7/20
15/15 [==============================] - 102s 7s/step - loss: 0.0152
Epoch 8/20
15/15 [==============================] - 102s 7s/step - loss: 0.0151
Epoch 9/20
15/15 [==============================] - 104s 7s/step - loss: 0.0151
Epoch 10/20
15/15 [==============================] - 103s 7s/step - loss: 0.0150
Epoch 11/20
15/15 [==============================] - 103s 7s/step - loss: 0.0149
Epoch 12/20
15/15 [==============================] - 102s 7s/step - loss: 0.0149
Epoch 13/20
15/15 [==================

In [13]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
4/4 [==============================] - 51s 7s/step - loss: 0.1335
Epoch 2/20
4/4 [==============================] - 28s 7s/step - loss: 0.1245
Epoch 3/20
4/4 [==============================] - 27s 7s/step - loss: 0.1073
Epoch 4/20
4/4 [==============================] - 27s 7s/step - loss: 0.0839
Epoch 5/20
4/4 [==============================] - 27s 7s/step - loss: 0.0597
Epoch 6/20
4/4 [==============================] - 27s 7s/step - loss: 0.0396
Epoch 7/20
4/4 [==============================] - 27s 7s/step - loss: 0.0274
Epoch 8/20
4/4 [==============================] - 27s 7s/step - loss: 0.0206
Epoch 9/20
4/4 [==============================] - 27s 7s/step - loss: 0.0172
Epoch 10/20
4/4 [==============================] - 27s 7s/step - loss: 0.0154
Epoch 11/20
4/4 [==============================] - 27s 7s/step - loss: 0.0142
Epoch 12/20
4/4 [==============================] - 27s 7s/step - loss: 0.0136
Epoch 13/20
4/4 [==============================] - 27s 7s/step - loss: 0.

In [14]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 122s 7s/step - loss: 0.0587
Epoch 2/20
14/14 [==============================] - 97s 7s/step - loss: 0.0314
Epoch 3/20
14/14 [==============================] - 98s 7s/step - loss: 0.0212
Epoch 4/20
14/14 [==============================] - 97s 7s/step - loss: 0.0175
Epoch 5/20
14/14 [==============================] - 96s 7s/step - loss: 0.0157
Epoch 6/20
14/14 [==============================] - 104s 7s/step - loss: 0.0146
Epoch 7/20
14/14 [==============================] - 98s 7s/step - loss: 0.0136
Epoch 8/20
14/14 [==============================] - 97s 7s/step - loss: 0.0128
Epoch 9/20
14/14 [==============================] - 96s 7s/step - loss: 0.0120
Epoch 10/20
14/14 [==============================] - 97s 7s/step - loss: 0.0113
Epoch 11/20
14/14 [==============================] - 97s 7s/step - loss: 0.0106
Epoch 12/20
14/14 [==============================] - 97s 7s/step - loss: 0.0100
Epoch 13/20
14/14 [============================

In [15]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
16/16 [==============================] - 144s 7s/step - loss: 0.0185
Epoch 2/20
16/16 [==============================] - 110s 7s/step - loss: 0.0108
Epoch 3/20
16/16 [==============================] - 118s 7s/step - loss: 0.0067
Epoch 4/20
16/16 [==============================] - 112s 7s/step - loss: 0.0054
Epoch 5/20
16/16 [==============================] - 112s 7s/step - loss: 0.0048
Epoch 6/20
16/16 [==============================] - 110s 7s/step - loss: 0.0045
Epoch 7/20
16/16 [==============================] - 111s 7s/step - loss: 0.0042
Epoch 8/20
16/16 [==============================] - 114s 7s/step - loss: 0.0039
Epoch 9/20
16/16 [==============================] - 111s 7s/step - loss: 0.0037
Epoch 10/20
16/16 [==============================] - 111s 7s/step - loss: 0.0035
Epoch 11/20
16/16 [==============================] - 112s 7s/step - loss: 0.0034
Epoch 12/20
16/16 [==============================] - 111s 7s/step - loss: 0.0033
Epoch 13/20
16/16 [==================

In [16]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
15/15 [==============================] - 136s 8s/step - loss: 0.1116
Epoch 2/20
15/15 [==============================] - 108s 7s/step - loss: 0.0884
Epoch 3/20
15/15 [==============================] - 106s 7s/step - loss: 0.0707
Epoch 4/20
15/15 [==============================] - 106s 7s/step - loss: 0.0634
Epoch 5/20
15/15 [==============================] - 108s 7s/step - loss: 0.0608
Epoch 6/20
15/15 [==============================] - 106s 7s/step - loss: 0.0598
Epoch 7/20
15/15 [==============================] - 123s 8s/step - loss: 0.0592
Epoch 8/20
15/15 [==============================] - 122s 8s/step - loss: 0.0587
Epoch 9/20
15/15 [==============================] - 123s 8s/step - loss: 0.0582
Epoch 10/20
15/15 [==============================] - 117s 8s/step - loss: 0.0578
Epoch 11/20
15/15 [==============================] - 109s 7s/step - loss: 0.0573
Epoch 12/20
15/15 [==============================] - 110s 7s/step - loss: 0.0567
Epoch 13/20
15/15 [==================

In [17]:
df = pd.DataFrame(res)
df.to_csv('DCAEOCSVM_ALL.csv')